In [1]:
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import json
from collections import Counter
import re
import spacy

%matplotlib inline
from sklearn import ensemble
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neural_network
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics
from time import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.utils import shuffle

import gensim
from gensim import corpora, models, similarities
import pyLDAvis.gensim as gensimvis
import pyLDAvis

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import tweepy
consumer_key = '66n3K6o0GfkbLO4QEoqRCA2pZ'
consumer_secret = 'T1fsUWei3jSvOehl9TmRqIuIhTGi0UEudOAeT04PMLcgodQzhS'
access_token = '86377594-cpO17ciBmTRbHzKD0AikvyIgJg7Ohfj5klfTogjVh'
access_token_secret = 'pMIxRtyha7XNeyBITlLwprYGwfl2DM4laCB2dUaDXCrCA'
 
def  twitter_Access():
     auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
     auth.set_access_token(access_token, access_token_secret)
     api = tweepy.API(auth, wait_on_rate_limit=True)
     return api
extract = twitter_Access()

In [3]:

#Twitter's API only allows for 200 records per call
TWEETS_PER_CALL = 200 

def save_tweets(screen_nm, tweets):
    # 'a' appends to the .json we wrote with 'w' above
    with open(f'C:\\Users\\Ankush\\Desktop\\{screen_nm}.json', 'a') as f:
        for tweet in tweets:
            json.dump([tweet.text, screen_nm], f); f.write('\n')

def get_tweets(screen_nm, desired_ct):
    
    #desired_ct is the number of tweets the user wants to include
    tweet_list = []

    #grab the current maximum tweet id for provided screen name, which will be the id of the first tweet stored in the object
    tweets = extract.user_timeline(screen_name = screen_nm,count=200)
    save_tweets(screen_nm, tweets)
    return tweet_list

In [4]:
trump = get_tweets('realDonaldTrump',200)
clinton = get_tweets('HillaryClinton', 200)
bernie = get_tweets('SenSanders', 200)
obama = get_tweets('BarackObama', 200)


In [5]:
handles = ['realDonaldTrump', 'HillaryClinton', 'SenSanders', 'BarackObama', 'MikeBloomberg']


In [6]:

#Combine all tweets into 1 list

full_tweets = []
for handle in handles:
    with open('C:\\Users\\Ankush\\Desktop\\{}.json'.format(handle)) as f: 
        full_tweets.append([json.loads(line) for line in f])
        
#Only want the first 2100 from each author (retweets caused the full amount for each author to be < 3200, and each has at least
#2100 non RTs)
all_tweets = []
for auth in full_tweets:
    all_tweets.extend(auth[:1000])
    
#Store in dataframe
tweet_df = pd.DataFrame(all_tweets, columns = ['Tweets', 'Author'])

In [12]:
tweet_df.head()

,Tweets,Author
0,....This will be remembered as one of the most...,realDonaldTrump
1,Senate Democrats just voted against legislatio...,realDonaldTrump
2,"If a deal is made with China, our great Americ...",realDonaldTrump
3,Heading over to Vietnam for my meeting with Ki...,realDonaldTrump
4,China Trade Deal (and more) in advanced stages...,realDonaldTrump


In [7]:
#tweet_full=tweet_df['Tweets'].replace(r'--',' ').astype('str')
tweet_full= ' '.join(tweet_df['Tweets'])

In [8]:
nlp = spacy.load('en')
tweet_full=nlp(tweet_full)

In [13]:
def bag_of_words(text):
    allwords = [token.lemma_ for token in text if not token.is_stop and not token.is_punct and not token.like_url and 
                not token.like_num and not token.is_space and token.is_ascii and len(token.lemma_) > 1]
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(1000)]

In [14]:
bow = bag_of_words(tweet_full)

#Let's clean this up a bit 
replace_words = [r'/n', r'\\n', r'\n', r'--', r'-', 'https:\/\/t\.co\/[\w\d]+', r'amp', r'&', r'w/', "'s"]
bow = [re.sub('|'.join(replace_words), '', word) for word in bow]

In [15]:
#Create dataframe with counts of our bag of words as features 
def bow_features(sentences, bow):
    
    partsofspeechlist = []
    ent_list = []
    
    for sent in sentences[0]:
        for token in sent:
                partsofspeechlist.append(token.pos_)

    parts = (set(partsofspeechlist))

    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns = set(list(bow) + list(parts)))
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, bow] = 0
    #df['tweet_length'] = 0
    #df['tweet_punct_count'] = 0 
    #df['upper_case'] = 0 
    df.loc[:, parts] = 0
    
    for i, sent in enumerate(sentences[0]):

        tweet_len = 0    
        num_punct = 0
        
        # Populate the row with word counts (only include the ones from our bag of words)
        words = [token.lemma_ for token in sent
                 if token.lemma_ in bow]
        
        #"words" collects all bow matches from each sentence
        for word in words:
            df.loc[i, word] += 1
        
        #Add part of speech counter and tweet length
        for token in sent:
            df.loc[i, token.pos_] += 1
            if not token.is_punct:
                tweet_len += 1
            else:
                num_punct += 1
            
        #Create a feature for all CAPS words  
           # if ((str(token).isupper()) & (len(str(token)) > 1)):
             #   df.loc[i, 'upper_case'] += 1


        #df.loc[i, 'tweet_length'] = tweet_len
        #df.loc[i, 'tweet_punct_count'] = num_punct

        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))

    return df

In [16]:
#Tokenize our tweets using spaCy (for more interesting features later on)
all_tweets_nlp = []
all_tweets_nlp.append([nlp(tweet[0]) for tweet in all_tweets])
all_tweets_nlp.append([nlp(tweet[1]) for tweet in all_tweets])

In [ ]:
df = bow_features(all_tweets_nlp, bow)
df.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


In [ ]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
X = np.array(df.drop(['text_sentence', 'text_source'],1))

#y needs to be str or int, cannot be spacy doc, so convert here
Y = df['text_source'].apply(lambda x: str(x))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)


#print('Training set score:', rfc.score(X_train, y_train))
#print('\nTest set score:', rfc.score(X_test, y_test))

In [ ]:
dt_grid={'max_depth':[3,4,5],'max_features': [5]}

grid_class_rfc=model_selection.GridSearchCV(rfc,dt_grid,cv=6)
grid_class_rfc.fit(X_train_pca, y_train_sample)
results = grid_class_rfc.cv_results_
print(results.get('params'))
print(results.get('mean_test_score'))
print(results.get('mean_train_score'))
print("Best Hyper Parameters:\n",grid_class_rfc.best_params_)
print(grid_class_rfc.best_score_)
final_model = grid_class_rfc.best_estimator_
print(final_model)

In [ ]:
#Fitting the best pararmeters to the model
rfc = ensemble.RandomForestClassifier(max_depth=5,max_features= 5)
rfc.fit(X_train_pca, y_train_sample)
y_pred_rfc = rfc.predict(X_train_pca)

# Cross Validaton


In [ ]:
print("Training data accuracy for Random Forest: ",cross_val_score(rfc, X_train_pca, y_train_sample, cv=10))
print("Testing data accuracy for Random Forest: ",cross_val_score(rfc, X_test_pca, y_test_sample, cv=10))

In [ ]:
from sklearn.metrics import roc_auc_score
print ("AUC Score for Random Forest :" ,roc_auc_score(y_train_sample, y_pred_rfc))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
print(confusion_matrix(y_train_sample, y_pred_rfc))
print('Accuracy Score for Random Forest :',accuracy_score(y_train_sample, y_pred_rfc))

In [ ]:

from sklearn.metrics import classification_report
print("Classification Report for Random Forest:",classification_report(y_train_sample, y_pred_rfc))

In [ ]:

#Confusion matrix for model random forest performance visualization


plt.figure(figsize=(10,6))
sns.heatmap(X.corr(),linewidth = 1,annot= True, annot_kws={"size": 9})
plt.title('Variable Correlation')